# Урок 6

## Домашнее задание.
Проект по итогам курса
По итогу занятий 1, 2, 3 и 4 у вас должна получиться таблица с данными об одном гене. Используя полученную информацию, проанализируйте её, проведите описательную статистику, постройте графики и прочее.

Чтобы посмотреть пример работы, обратитесь к разделу 4 на kaggle.com. Полученные результаты предоставляются в формате .ipynb.

1. Оценить частоту мутаций в популяциях по выбранному гену.
2. Оценить частоту мутаций в разрезе связанных заболеваний.

## Задание выполнено командой M4 Team.
__Состав команды:__  
 - Мгер Парунакян
 - Михаил Демин
 - Марат Закиров
 - Марина Дружинина

In [ ]:
!pip install biopython
from Bio import Entrez
Entrez.email = "A.N.Other@example.com"
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Colab Notebooks/GB/19_6_files'

/content/drive/MyDrive/Colab Notebooks/GB/19_6_files


In [ ]:
# Получаем информацию из базы данных SNP для гена FGFR2
gene_name = "FGFR2"
org = "homo sapiens"

# search ClinVar for variants in the gene
try:
    handle = Entrez.esearch(db="snp", term=f"{gene_name}[Gene Name]  AND {org}[Organism] AND (pathogenic[Clinical_Significance] OR likely pathogenic[Clinical_Significance] OR pathogenic likely pathogenic[Clinical_Significance])", retmax=100)
    
except urllib.error.HTTPError as e:
    if e.code == 429:
        print("Too many requests. Waiting for 10 seconds...")
        time.sleep(10)
        handle = Entrez.esearch(db="snp", term=f"{gene_name}[Gene Name]  AND {org}[Organism] AND (pathogenic[Clinical_Significance] OR likely pathogenic[Clinical_Significance] OR pathogenic likely pathogenic[Clinical_Significance])", retmax=100)
    else:
        raise e
record = Entrez.read(handle)
handle.close()
rs_list = record['IdList']
len(rs_list)

96

In [ ]:
rs_list

['2134256765', '2134256250', '2134253609', '2133823028', '1850314485', '1850299732', '1850289942', '1847510893', '1847174277', '1845559552', '1589828632', '1589722765', '1564919048', '1564875577', '1554930637', '1554928978', '1554928884', '1554927408', '1554917471', '1554914180', '1554907364', '1554907337', '1434545235', '1358919643', '1274989878', '1064796452', '1064796413', '1057520044', '1057520029', '1057520028', '1057520027', '1057519901', '1057519900', '1057519854', '1057519800', '1057519799', '1057519798', '1057519797', '1057519796', '1057519795', '1057519791', '1057519047', '1057519046', '1057519045', '1057519044', '1057519043', '1057519042', '1057519041', '1057519040', '1057519039', '1057519038', '1057519037', '1057519036', '886037837', '879253721', '879253720', '879253719', '879253718', '777169135', '776587763', '387907372', '387906678', '387906677', '387906676', '281865420', '121918510', '121918509', '121918508', '121918507', '121918506', '121918505', '121918504', '121918503

In [ ]:
# Получаем информацию по каждой мутации из списка
def get_snp_rs_info(rs_list):
  snp_rs_info = []
  for k, i in enumerate(rs_list):
    print(k)
    try:
        handle = Entrez.esearch(db="snp", term="rs" + i, format='gb')
    except urllib.error.HTTPError as e:
        if e.code == 429:
            print("Too many requests. Waiting for 10 seconds...")
            time.sleep(10)
            handle = Entrez.esearch(db="snp", term="rs" + i, format='gb')
        else:
            raise e
    record = Entrez.read(handle)
    handle.close()
    idlist = record["IdList"]
    if len(idlist) > 0:
        handle = Entrez.esummary(db="snp", id=",".join(idlist))
        result = Entrez.read(handle)
        handle.close()
        snp_info = result['DocumentSummarySet']['DocumentSummary'][0]
        snp_rs_info.append({key: snp_info[key] for key in snp_info.keys()})
        #snp_rs_info.append({'SNP_ID':f"rs{snp_info['SNP_ID']}", 'GENES':snp_info['GENES'], 'ACC':snp_info['ACC'], 'SPDI':snp_info['SPDI'], 'ALLELE':" > ".join(snp_info['SPDI'].split(':')[2:]), 'CLINICAL_SIGNIFICANCE':snp_info['CLINICAL_SIGNIFICANCE'], 'CLINICAL_SORT':snp_info['CLINICAL_SORT']})
  return snp_rs_info
  #return result

In [ ]:
df_FGFR2_snp = pd.DataFrame(get_snp_rs_info(rs_list))
df_FGFR2_snp.to_csv(f'FGFR2_snp.csv', sep=';', index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [ ]:
# Получаем информацию по популяциям
# from multiprocessing import Pool

def get_population(rs_list):
    df_base = pd.DataFrame(columns=['RS_Num', 'Study', 'Population', 'Group', 'Samplesize', 'Ref Allele', 'Alt Allele'])
    q = 1
    for i in rs_list:
        i = "rs" + i
        print(q, i)
        q += 1
        url = 'https://www.ncbi.nlm.nih.gov/snp/'+ i

        # Send a request to the website and get the HTML content
        response = requests.get(url)
        html_content = response.content

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the table containing the variant information
        table = soup.find(id = 'dbsnp_freq_datatable')
        # Get the rows of the table
        try:
            rows = table.find_all('tr')
            
            # Iterate through the rows and extract the table elements
            table_elements = []
            for row in rows:
                columns = row.find_all('td')
                row_data = []
                for column in columns:
                    row_data.append(column.get_text().strip())
                table_elements.append(row_data)

            # Print the table elements
            rows = []
            for row in table_elements:
                row.insert(0, i)
                rows.append(row)
            rows.pop(0)
            df = pd.DataFrame(rows, columns=['RS_Num', 'Study', 'Population', 'Group', 'Samplesize', 'Ref Allele', 'Alt Allele'])
            df_base = pd.concat([df_base, df])
        except Exception as ex:
            print(ex)
            continue
    return df_base.reset_index()

# if __name__ == '__main__':
#   pool = Pool(processes=16)
#   results = pool.map(get_population, rs_list)
#   pool.close()
#   pool.join()

In [ ]:
df_FGFR2_pop = get_population(rs_list)

1 rs2134256765
'NoneType' object has no attribute 'find_all'
2 rs2134256250
'NoneType' object has no attribute 'find_all'
3 rs2134253609
'NoneType' object has no attribute 'find_all'
4 rs2133823028
'NoneType' object has no attribute 'find_all'
5 rs1850314485
6 rs1850299732
'NoneType' object has no attribute 'find_all'
7 rs1850289942
'NoneType' object has no attribute 'find_all'
8 rs1847510893
'NoneType' object has no attribute 'find_all'
9 rs1847174277
'NoneType' object has no attribute 'find_all'
10 rs1845559552
'NoneType' object has no attribute 'find_all'
11 rs1589828632
'NoneType' object has no attribute 'find_all'
12 rs1589722765
'NoneType' object has no attribute 'find_all'
13 rs1564919048
'NoneType' object has no attribute 'find_all'
14 rs1564875577
15 rs1554930637
'NoneType' object has no attribute 'find_all'
16 rs1554928978
'NoneType' object has no attribute 'find_all'
17 rs1554928884
'NoneType' object has no attribute 'find_all'
18 rs1554927408
19 rs1554917471
'NoneType' obje

In [ ]:
df_FGFR2_pop.to_csv('FGFR2_pop.csv', sep=';', index=False)

In [ ]:
# Получаем информацию по болезням связанным с мутацией
# from multiprocessing import Pool

def get_rs_disease(rs_list):
    df_base = pd.DataFrame(columns=['SNP_ID_RS', 'ClinVar_Accession', 'Disease_Names', 'Clinical_Significance', 'Mutation_type'])
    q = 1
    for i in rs_list:
        i = "rs" + i
        print(q, i)
        q += 1
        url = 'https://www.ncbi.nlm.nih.gov/snp/'+ i # + '#clinical_significance'

        # Send a request to the website and get the HTML content
        response = requests.get(url)
        html_content = response.content

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find the table containing the variant information
        table = soup.find(id = 'clinical_significance_datatable')

        # Find the <span> element with the clinical significance value using its XPath
        mut_type_dl = soup.find_all('dl', {'class': 'usa-width-one-half'})[1].find_all('dd')[1].find('span')
        # dd = consequence_dl[1].find_all('dd')[1].find('span')
        # span = dd[1].find('span')
        
        # Get the rows of the table
        try:
            rows = table.find_all('tr')
            
            # Iterate through the rows and extract the table elements
            table_elements = []
            for row in rows:
                columns = row.find_all('td')
                row_data = []
                for column in columns:
                    row_data.append(column.get_text().strip())
                table_elements.append(row_data)

            # Print the table elements
            rows = []
            for row in table_elements:
                row.insert(0, i)
                rows.append(row)
            rows.pop(0)
            df = pd.DataFrame(rows, columns=['SNP_ID_RS', 'ClinVar_Accession', 'Disease_Names', 'Clinical_Significance'])
            mut_type = str(mut_type_dl.text).split(': ')[1]
            df['Mutation_type'] = mut_type
            df_base = pd.concat([df_base, df])
        except Exception as ex:
            print(ex)
            continue
    return df_base.reset_index()

# if __name__ == '__main__':
#   pool = Pool(processes=16)
#   results = pool.map(get_population, rs_list)
#   pool.close()
#   pool.join()

In [ ]:
df_FGFR2_dis = get_rs_disease(rs_list)

1 rs2134256765
2 rs2134256250
3 rs2134253609
4 rs2133823028
5 rs1850314485
6 rs1850299732
7 rs1850289942
8 rs1847510893
9 rs1847174277
10 rs1845559552
11 rs1589828632
12 rs1589722765
13 rs1564919048
14 rs1564875577
15 rs1554930637
16 rs1554928978
17 rs1554928884
18 rs1554927408
19 rs1554917471
20 rs1554914180
21 rs1554907364
22 rs1554907337
23 rs1434545235
24 rs1358919643
25 rs1274989878
26 rs1064796452
27 rs1064796413
28 rs1057520044
29 rs1057520029
30 rs1057520028
31 rs1057520027
32 rs1057519901
33 rs1057519900
34 rs1057519854
35 rs1057519800
36 rs1057519799
37 rs1057519798
38 rs1057519797
39 rs1057519796
40 rs1057519795
41 rs1057519791
42 rs1057519047
43 rs1057519046
44 rs1057519045
45 rs1057519044
46 rs1057519043
47 rs1057519042
48 rs1057519041
49 rs1057519040
50 rs1057519039
51 rs1057519038
52 rs1057519037
53 rs1057519036
54 rs886037837
55 rs879253721
56 rs879253720
57 rs879253719
58 rs879253718
59 rs777169135
60 rs776587763
61 rs387907372
62 rs387906678
63 rs387906677
64 rs387906

In [ ]:
df_FGFR2_dis.to_csv('FGFR2_dis.csv', sep=';', index=False)

In [ ]:
# part = 200
# parts = int(len(rs_list) / part)
# for i in range(int(parts)):
#     df = get_population(rs_list[i*part:(i+1)*part])
#     if i == parts-1:
#         df = get_population(rs_list[(i+1)*part:])
#     del df['index']
#     df.to_csv(f'df{(i+1)*part}.csv', sep=';')

In [ ]:
# s=0
# part = 200
# parts = len(rs_list) / part
# for i in range(int(parts)):
#     df = get_population(rs_list[s:part])
#     s=part
#     part+=200
#     del df['index']
#     df.to_csv(f'df{s}.csv', sep=';')

In [ ]:
# # Соединим все файлы в один
  
# path = "/content/df" #hw_6_gene"  
# # Change the directory
# os.chdir(path)

In [ ]:
# def read_text_file(file_path,  sep=','):
#     with open(file_path, 'r') as f:
#         return pd.read_csv(f, sep=sep, index_col= 0)

In [ ]:
# # iterate through all file
# df = pd.DataFrame(columns=['RS_Num', 'Study', 'Population', 'Group', 'Samplesize', 'Ref Allele', 'Alt Allele'])
# for file in os.listdir():
#     # Check whether file is in text format or not
#     if file.endswith(".csv"):
#         file_path = f"{path}//{file}"
#         df_part = read_text_file(file_path, sep=';')
#         df = pd.concat([df, df_part])

In [ ]:
# df_part = read_text_file('df_end.csv', sep=';')
# df = pd.concat([df, df_part])

In [ ]:
# df_part = read_text_file('/content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/df_all.csv', sep=';')
# df = read_text_file('/content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/df_population', sep=';')
# df = pd.concat([df, df_part])
# df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
# df.to_csv('/content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/df_fgfr2_population.csv', sep=';')

In [ ]:
df_pop = pd.read_csv('FGFR2_pop.csv', sep=';')
df_pop.head(1)

,index,RS_Num,Study,Population,Group,Samplesize,Ref Allele,Alt Allele
0,0,rs1850314485,TopMed,Global,Study-wide,264690,C=0.999996,T=0.000004


In [ ]:
df_all = pd.read_csv('FGFR2_snp.csv', sep=';')
df_all.head(1)

,SNP_ID,ALLELE_ORIGIN,GLOBAL_MAFS,GLOBAL_POPULATION,GLOBAL_SAMPLESIZE,SUSPECTED,CLINICAL_SIGNIFICANCE,GENES,ACC,CHR,...,ALLELE,SNP_CLASS,CHRPOS,CHRPOS_PREV_ASSM,TEXT,SNP_ID_SORT,CLINICAL_SORT,CITED_SORT,CHRPOS_SORT,MERGED_SORT
0,2134256765,NaN,[],NaN,0,NaN,likely-pathogenic,"[{'NAME': 'FGFR2', 'GENE_ID': '2263'}]",NC_000010.11,10,...,N,ins,10:121517381,10:123276896,NaN,2134256765,1,NaN,121517381,0


In [ ]:
df_dis = pd.read_csv('FGFR2_dis.csv', sep=';')
df_dis.head(1)

,index,SNP_ID_RS,ClinVar_Accession,Disease_Names,Clinical_Significance,Mutation_type
0,0,rs2134256765,RCV002224437.1,not provided,Likely-Pathogenic,Inframe Indel


In [ ]:
del df_pop['index']
df_pop.rename(columns = {'RS_Num':'SNP_ID_RS'}, inplace = True )
df_pop.head(1)

,SNP_ID_RS,Study,Population,Group,Samplesize,Ref Allele,Alt Allele
0,rs1850314485,TopMed,Global,Study-wide,264690,C=0.999996,T=0.000004


In [ ]:
df_all['SNP_ID_RS'] = df_all['SNP_ID'].apply(lambda x: 'rs' + str(x))
df_all.head(1)

,SNP_ID,ALLELE_ORIGIN,GLOBAL_MAFS,GLOBAL_POPULATION,GLOBAL_SAMPLESIZE,SUSPECTED,CLINICAL_SIGNIFICANCE,GENES,ACC,CHR,...,SNP_CLASS,CHRPOS,CHRPOS_PREV_ASSM,TEXT,SNP_ID_SORT,CLINICAL_SORT,CITED_SORT,CHRPOS_SORT,MERGED_SORT,SNP_ID_RS
0,2134256765,NaN,[],NaN,0,NaN,likely-pathogenic,"[{'NAME': 'FGFR2', 'GENE_ID': '2263'}]",NC_000010.11,10,...,ins,10:121517381,10:123276896,NaN,2134256765,1,NaN,121517381,0,rs2134256765


In [ ]:
df_2 = df_dis.merge(df_all, how='left', on='SNP_ID_RS')
df_2.head(1)

,index,SNP_ID_RS,ClinVar_Accession,Disease_Names,Clinical_Significance,Mutation_type,SNP_ID,ALLELE_ORIGIN,GLOBAL_MAFS,GLOBAL_POPULATION,...,ALLELE,SNP_CLASS,CHRPOS,CHRPOS_PREV_ASSM,TEXT,SNP_ID_SORT,CLINICAL_SORT,CITED_SORT,CHRPOS_SORT,MERGED_SORT
0,0,rs2134256765,RCV002224437.1,not provided,Likely-Pathogenic,Inframe Indel,2134256765,NaN,[],NaN,...,N,ins,10:121517381,10:123276896,NaN,2134256765,1,NaN,121517381,0


In [ ]:
df_2.to_parquet('FGFR2_disease_dataset.parquet.gzip', compression='gzip')

In [ ]:
df_3 = df_pop.merge(df_all, how='right', on='SNP_ID_RS')
df_3.head(1)

,SNP_ID_RS,Study,Population,Group,Samplesize,Ref Allele,Alt Allele,SNP_ID,ALLELE_ORIGIN,GLOBAL_MAFS,...,ALLELE,SNP_CLASS,CHRPOS,CHRPOS_PREV_ASSM,TEXT,SNP_ID_SORT,CLINICAL_SORT,CITED_SORT,CHRPOS_SORT,MERGED_SORT
0,rs2134256765,NaN,NaN,NaN,NaN,NaN,NaN,2134256765,NaN,[],...,N,ins,10:121517381,10:123276896,NaN,2134256765,1,NaN,121517381,0


In [ ]:
df_3.to_parquet('FGFR2_population_dataset.parquet.gzip', compression='gzip')

In [ ]:
# df2.to_csv(f'/content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/M4team_dataset.csv', sep=';')

In [ ]:
# !zip -r /content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/M4team_dataset.csv /content/M4team_dataset.zip

In [ ]:
# df2.to_parquet('/content/drive/MyDrive/Colab_Notebooks/microbio/hw_6/M4team_dataset.parquet.gzip', compression='gzip')

In [ ]:
df2 = pd.read_parquet('FGFR2_disease_dataset.parquet.gzip')